<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/modeling/multiclass_classification/CA_NY_TX_Classification_after_CDC_GuidelineLabeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal:

* This colab is for multi class classification of CDC guidelined labeled data for combined datasets and maryland inertia data for 3 states: California, New York and Texas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install wget
import wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=eae7e36d974112d89964e6ca70f1e491509e8f996ce3228ff2717439711332b6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
import seaborn as sns
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
!ls  '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021'

 CA_alldatasets_Feb05.csv
 CA_alldatasets_Jan13.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb05.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb06.csv
 CA_CombinedData_Labeled_CDCGuideline_Jan15.csv
 CA_combined_Feb04.csv
 CA_CovidDataJan10.csv
 CA_GoogleMobilityDataJan10.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb05.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb06.csv
 CA_MarylandData_Labeled_CDCGuideline_Jan15.csv
 CA_processedMarylandData_Feb04.csv
 CA_processedMarylandData_Feb05.csv
 CA_processedMarylandData_Jan11.csv
 CA_SocialDistancingDataJan10.csv
 CA_socialDistancingInertiaData_Feb04.csv
 computed_CA_mobility_data.csv
 data-mask_for_all.csv
 election_results_2021.csv
 maryland_inertia_data_allstates_Feb04.csv
 maryland_inertia_data_allstates_Feb05.csv
 maryland_inertia_data_allstates_Jan11.csv
 maryland_inertia_raw_dec28_2020.csv
 Mask_Rule_Feb05.csv
 Mask_Rule_Jan10.csv
 NY_alldatasets_Jan13.csv
 NY_CombinedData_Labeled_CDCGuideline_Feb07.csv
 NY_CombinedData_Labeled_CDCGuideli

# File Location

In [6]:
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'

# Load Combined Labeled Dataset

### 1.CA

In [7]:
CA_combined_data = pd.read_csv(location+'CA_CombinedData_Labeled_CDCGuideline_Feb06.csv',low_memory=False)
CA_combined_data.head()

,Date,Province_State,FIPS,County Name,mask_rule_severity,PopDensity,population,mask_rule_active,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,New Cases/100k population,rolling_avg_new_cases/100k,New cases,growth_label
0,2020-03-01,California,6001,Alameda County,0.0,860.757075,1671329,0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,0.000000,0.0,0.0,minimal
1,2020-03-02,California,6001,Alameda County,0.0,860.757075,1671329,0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,0.000000,0.0,0.0,minimal
2,2020-03-03,California,6001,Alameda County,0.0,860.757075,1671329,0,6.271,100,5.0,12.0,41.0,0.0,1.0,-1.0,0.059833,0.0,1.0,minimal
3,2020-03-04,California,6001,Alameda County,0.0,860.757075,1671329,0,6.209,99,1.0,6.0,23.0,-3.0,1.0,0.0,0.000000,0.0,0.0,minimal
4,2020-03-05,California,6001,Alameda County,0.0,860.757075,1671329,0,6.654,106,1.0,6.0,12.0,-5.0,-1.0,1.0,0.000000,0.0,0.0,minimal


### 2.NY

In [8]:
NY_combined_data = pd.read_csv(location+'NY_CombinedData_Labeled_CDCGuideline_Feb07.csv',low_memory=False)
NY_combined_data.head()

,Date,Province_State,FIPS,County Name,population,mask_rule_active,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,New Cases/100k population,rolling_avg_new_cases/100k,New cases,growth_label
0,2020-03-01,New York,36001,Albany County,305506,0,2.616,54,10.0,13.0,19.0,4.0,7.0,-1.0,0.0,0.0,0.0,minimal
1,2020-03-02,New York,36001,Albany County,305506,0,4.680,97,11.0,15.0,29.0,5.0,3.0,0.0,0.0,0.0,0.0,minimal
2,2020-03-03,New York,36001,Albany County,305506,0,4.778,100,8.0,15.0,41.0,8.0,3.0,-1.0,0.0,0.0,0.0,minimal
3,2020-03-04,New York,36001,Albany County,305506,0,4.486,93,7.0,8.0,6.0,2.0,3.0,0.0,0.0,0.0,0.0,minimal
4,2020-03-05,New York,36001,Albany County,305506,0,5.129,107,5.0,13.0,18.0,2.0,3.0,-1.0,0.0,0.0,0.0,minimal


### 3.TX

In [9]:
TX_combined_data = pd.read_csv(location+'TX_CombinedData_Labeled_CDCGuideline_Feb07.csv',low_memory=False)
TX_combined_data.head()

,Date,Province_State,FIPS,County Name,population,mask_rule_active,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,New Cases/100k population,rolling_avg_new_cases/100k,New cases,growth_label
0,2020-03-01,Texas,48001,Anderson County,57735,0,8.910,79,16.0,15.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,minimal
1,2020-03-02,Texas,48001,Anderson County,57735,0,10.708,95,14.0,7.0,0.0,0.0,2.0,-1.0,0.0,0.0,0.0,minimal
2,2020-03-03,Texas,48001,Anderson County,57735,0,12.118,107,21.0,8.0,0.0,0.0,2.0,-2.0,0.0,0.0,0.0,minimal
3,2020-03-04,Texas,48001,Anderson County,57735,0,9.156,81,12.0,5.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,minimal
4,2020-03-05,Texas,48001,Anderson County,57735,0,11.230,100,18.0,13.0,0.0,0.0,3.0,-3.0,0.0,0.0,0.0,minimal


### Merge 3 states

In [10]:
states_data = [CA_combined_data, NY_combined_data, TX_combined_data]

Combined_data = pd.concat(states_data)

In [11]:
Combined_data.head(2)

,Date,Province_State,FIPS,County Name,mask_rule_severity,PopDensity,population,mask_rule_active,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,New Cases/100k population,rolling_avg_new_cases/100k,New cases,growth_label
0,2020-03-01,California,6001,Alameda County,0.0,860.757075,1671329,0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,0.0,0.0,0.0,minimal
1,2020-03-02,California,6001,Alameda County,0.0,860.757075,1671329,0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,0.0,0.0,0.0,minimal


### Categorical code

In [12]:
Combined_data["growth_label"] = Combined_data["growth_label"].astype('category')
Combined_data["growth_label_cat"] = Combined_data["growth_label"].cat.codes

## X_col + Y_col

In [13]:
Combined_data.columns

Index(['Date', 'Province_State', 'FIPS', 'County Name', 'mask_rule_severity',
       'PopDensity', 'population', 'mask_rule_active', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'New Cases/100k population',
       'rolling_avg_new_cases/100k', 'New cases', 'growth_label',
       'growth_label_cat'],
      dtype='object')

In [14]:
# don't use it; just for reference - version 1
old_col =['FIPS','retail and recreation',
       'grocery and pharmacy', 'parks', 'transit stations', 'workplaces',
       'residential', 'driving', 'm50', 'm50_index', 'population_density',
       'mask_rule_active', 'mask_wearing_percent']

In [15]:
Y_col = Combined_data['growth_label_cat']
X_col = Combined_data[['FIPS','mask_rule_active', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']] #'New cases' # 'population_density'

## Split data

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X_col, Y_col, test_size=0.2, random_state=1)

## Automatic Model Selection

In [17]:
num_folds = 10
seed = 7
scoring = 'accuracy'

models = []
models.append(('LR' , LogisticRegression()))
models.append(('LDA' , LinearDiscriminantAnalysis()))
models.append(('KNN' , KNeighborsClassifier()))
models.append(('CART' , DecisionTreeClassifier()))
models.append(('NB' , GaussianNB()))
models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
models.append(('XGBoost', XGBClassifier()))

In [18]:
results = []
names = []

'''
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) '''

for name, model in models:
    clf = model
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accu_score = accuracy_score(Y_test, Y_pred)
    print(name + ": " + str(accu_score))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LR: 0.5079000287273772
LDA: 0.5581729388106866
KNN: 0.6469405343292157
CART: 0.6508474576271186
NB: 0.461304222924447
SVM: 0.5157713300775639
RF: 0.7315713875323183
XGBoost: 0.6667624245906348


## Result 
Previous Result:
LR: 0.5195058891123241
LDA: 0.6461936225222638
KNN: 0.649008905486929
CART: 0.6599253088193048
NB: 0.5505314564780236
SVM: 0.49330652111462225
RF: 0.7467394426888825
XGBoost: 0.6899741453605286

# Load Inertia Labeled Dataset

### 1.CA

In [19]:
CA_maryland_sd_inertia = pd.read_csv(location+'CA_MarylandData_Labeled_CDCGuideline_Feb06.csv',low_memory=False)
CA_maryland_sd_inertia.head()

,County Name,State,mask_rule_severity,PopDensity,population,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,index,State_Name,countyFIPS,growth_label
0,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,50,32,2.41,26.1,0.4,38.0,0.20,2.21,0,1666753,-21.7,2020-01-01,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0,0.0,67518.0,California,6001.0,minimal
1,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,24,22,3.17,27.9,0.4,38.5,0.50,2.66,0,1666753,-5.8,2020-01-02,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0,0.0,67519.0,California,6001.0,minimal
2,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,21,21,3.28,27.2,0.4,41.0,0.51,2.78,0,1666753,-1.5,2020-01-03,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0,0.0,67520.0,California,6001.0,minimal
3,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,34,25,3.03,25.8,0.3,41.1,0.26,2.76,0,1666753,-2.2,2020-01-04,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0,0.0,67521.0,California,6001.0,minimal
4,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,44,29,2.72,24.5,0.4,35.8,0.24,2.48,0,1666753,-12.1,2020-01-05,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,1.3,3.9,5.7,0.3,0.0,0.0,0.0,67522.0,California,6001.0,minimal


### 2.NY

In [20]:
NY_maryland_sd_inertia = pd.read_csv(location+'NY_MarylandData_Labeled_CDCGuideline_Feb07.csv',low_memory=False)
NY_maryland_sd_inertia.head()

,County Name,State,population,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,countyFIPS,growth_label
0,Albany County,NY,305506,36001,Albany County,36,54,35,3.01,34.6,3.0,34.9,0.38,2.63,0,307117,-18.8,2020-01-01,6.06,22,64535,11.7,5.8,48.44,577,465,151,2.74,0.22,0.029,0.0,0.0,0,63.5,0.0,0.0,0.0,0,0,0.0,0,2.3,3.8,4.2,0.7,0.0,0.0,0.0,36001.0,minimal
1,Albany County,NY,305506,36001,Albany County,36,19,20,4.31,39.4,2.5,34.8,1.00,3.31,0,307117,2.3,2020-01-02,6.06,22,64535,11.7,5.8,48.44,577,465,151,2.74,0.22,0.029,0.0,0.0,0,63.5,0.0,0.0,0.0,0,0,0.0,0,2.3,3.8,4.2,0.7,0.0,0.0,0.0,36001.0,minimal
2,Albany County,NY,305506,36001,Albany County,36,17,20,4.47,39.5,2.4,41.4,1.01,3.46,0,307117,6.9,2020-01-03,6.06,22,64535,11.7,5.8,48.44,577,465,151,2.74,0.22,0.029,0.0,0.0,0,63.5,0.0,0.0,0.0,0,0,0.0,0,2.3,3.8,4.2,0.7,0.0,0.0,0.0,36001.0,minimal
3,Albany County,NY,305506,36001,Albany County,36,35,25,3.78,37.3,2.6,36.0,0.45,3.33,0,307117,2.9,2020-01-04,6.06,22,64535,11.7,5.8,48.44,577,465,151,2.74,0.22,0.029,0.0,0.0,0,63.5,0.0,0.0,0.0,0,0,0.0,0,2.3,3.8,4.2,0.7,0.0,0.0,0.0,36001.0,minimal
4,Albany County,NY,305506,36001,Albany County,36,46,30,3.30,34.8,2.8,36.5,0.39,2.90,0,307117,-10.4,2020-01-05,6.06,22,64535,11.7,5.8,48.44,577,465,151,2.74,0.22,0.029,0.0,0.0,0,63.5,0.0,0.0,0.0,0,0,0.0,0,1.2,3.8,4.2,0.7,0.0,0.0,0.0,36001.0,minimal


### 3.TX

In [21]:
TX_maryland_sd_inertia = pd.read_csv(location+'TX_MarylandData_Labeled_CDCGuideline_Feb07.csv',low_memory=False)
TX_maryland_sd_inertia.head()

,County Name,State,population,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,countyFIPS,growth_label
0,Anderson County,TX,57735,48001,Anderson County,48,45,24,2.58,20.1,0.4,34.1,0.30,2.28,0,58057,-22.8,2020-01-01,0.22,20,43355,21.0,17.5,61.2,54,12,95,2.82,0.3,0.04,0.0,0.0,0,47.7,0.0,0.0,0.0,0,0,0.0,0,0.4,3.5,4.6,0.2,0.0,0.0,0.0,48001.0,minimal
1,Anderson County,TX,57735,48001,Anderson County,48,16,16,3.36,21.3,0.3,42.7,0.53,2.82,0,58057,-4.5,2020-01-02,0.22,20,43355,21.0,17.5,61.2,54,12,95,2.82,0.3,0.04,0.0,0.0,0,47.7,0.0,0.0,0.0,0,0,0.0,0,0.4,3.5,4.6,0.2,0.0,0.0,0.0,48001.0,minimal
2,Anderson County,TX,57735,48001,Anderson County,48,10,13,3.65,23.5,0.5,47.7,0.55,3.10,0,58057,4.9,2020-01-03,0.22,20,43355,21.0,17.5,61.2,54,12,95,2.82,0.3,0.04,0.0,0.0,0,47.7,0.0,0.0,0.0,0,0,0.0,0,0.4,3.5,4.6,0.2,0.0,0.0,0.0,48001.0,minimal
3,Anderson County,TX,57735,48001,Anderson County,48,20,17,3.33,25.5,0.5,46.2,0.40,2.93,0,58057,-0.8,2020-01-04,0.22,20,43355,21.0,17.5,61.2,54,12,95,2.82,0.3,0.04,0.0,0.0,0,47.7,0.0,0.0,0.0,0,0,0.0,0,0.4,3.5,4.6,0.2,0.0,0.0,0.0,48001.0,minimal
4,Anderson County,TX,57735,48001,Anderson County,48,32,21,2.93,21.4,0.6,42.7,0.36,2.57,0,58057,-13.0,2020-01-05,0.22,20,43355,21.0,17.5,61.2,54,12,95,2.82,0.3,0.04,0.0,0.0,0,47.7,0.0,0.0,0.0,0,0,0.0,0,0.7,3.5,4.6,0.2,0.0,0.0,0.0,48001.0,minimal


### Merge 3 states

In [22]:
maryland_states_data = [CA_maryland_sd_inertia, NY_maryland_sd_inertia, TX_maryland_sd_inertia]

maryland_sd_inertia = pd.concat(maryland_states_data)

In [23]:
maryland_sd_inertia.head(2)

,County Name,State,mask_rule_severity,PopDensity,population,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,index,State_Name,countyFIPS,growth_label
0,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,50,32,2.41,26.1,0.4,38.0,0.2,2.21,0,1666753,-21.7,2020-01-01,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0,0.0,67518.0,California,6001.0,minimal
1,Alameda County,CA,0.0,860.757075,1671329,6001,Alameda County,6,24,22,3.17,27.9,0.4,38.5,0.5,2.66,0,1666753,-5.8,2020-01-02,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0,0.0,67519.0,California,6001.0,minimal


### Categorical code

In [24]:
maryland_sd_inertia["growth_label"] = maryland_sd_inertia["growth_label"].astype('category')
maryland_sd_inertia["growth_label_cat"] = maryland_sd_inertia["growth_label"].cat.codes

In [25]:

pd.unique(maryland_sd_inertia[['growth_label', 'growth_label_cat']].values.ravel('K'))

array(['minimal', 'moderate', 'substantial', 'widespread', 0, 1, 2, 3],
      dtype=object)

## X_col + Y_col

In [26]:
#not using this; just for version-1 reference
old_col =['CTFIPS','Social distancing index', '% staying home', 'Trips/person',
       '% out-of-county trips', '% out-of-state trips', 'Miles/person',
       'Work trips/person', 'Non-work trips/person','Population','Transit mode share',
       '% people older than 60', 'Median income', '% African Americans',
       '% Hispanic Americans', '% Male', 'Population density',
       'Employment density', '# hot spots/1000 people',
       'Hospital beds/1000 people', 'ICUs/1000 people',
       '# contact tracing workers/1000 people','% hospital bed utilization', 'Testing capacity',
       'Tests done/1000 people', '% ICU utilization', 'Ventilator shortage',
       'Imported COVID cases', 'COVID exposure/1000 people',
       '#days: decreasing ILI cases', 'Unemployment claims/1000 people',
       'Unemployment rate', '% working from home', 'Cumulative inflation rate']

In [27]:
maryland_sd_inertia.columns

Index(['County Name', 'State', 'mask_rule_severity', 'PopDensity',
       'population', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'date', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Population.density',
       'Employment.density', 'X..hot.spots.1000.people',
       'Hospital.beds.1000.people', 'ICUs.1000.people',
       'X..contact.tracing.workers.1000.people', 'New.cases.1000.people',
       'Active.cases.1000.people', 'X.days..decreasing.COVID.cases',
       'X..hospital.bed.utilization', 'Testing.capacity',
       'Tests.done.1000.people', 'X..ICU.utilization', 'Ventilator.shortage',
       'Imported.COVID.cases', 'COVID.exposure.1000.peop

In [28]:
Y_col = maryland_sd_inertia['growth_label_cat']
X_col = maryland_sd_inertia[['CTFIPS','Social.distancing.index', 'X..staying.home', 'Trips.person',
       'X..out.of.county.trips', 'X..out.of.state.trips', 'Miles.person',
       'Work.trips.person', 'Non.work.trips.person', 'New.COVID.cases',
       'Population', 'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Population.density',
       'Employment.density', 'X..hot.spots.1000.people',
       'Hospital.beds.1000.people', 'ICUs.1000.people',
       'X..contact.tracing.workers.1000.people', 'New.cases.1000.people',
       'Active.cases.1000.people', 'X.days..decreasing.COVID.cases',
       'X..hospital.bed.utilization', 'Testing.capacity',
       'Tests.done.1000.people', 'X..ICU.utilization', 'Ventilator.shortage',
       'Imported.COVID.cases', 'COVID.exposure.1000.people',
       'X.days..decreasing.ILI.cases', 'Unemployment.claims.1000.people',
       'Unemployment.rate', 'X..working.from.home','Cumulative.inflation.rate']]

## Split data

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X_col, Y_col, test_size=0.2, random_state=1)

## Automatic Model Selection- Model with Evaluation

In [30]:
num_folds = 10
seed = 7
scoring = 'accuracy'

models = []
models.append(('LR' , LogisticRegression()))
models.append(('LDA' , LinearDiscriminantAnalysis()))
models.append(('KNN' , KNeighborsClassifier()))
models.append(('CART' , DecisionTreeClassifier()))
models.append(('NB' , GaussianNB()))
#models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
models.append(('XGBoost', XGBClassifier()))

In [31]:
results = []
names = []

'''
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) '''

for name, model in models:
    clf = model
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accu_score = accuracy_score(Y_test, Y_pred)
    print(name + ": " + str(accu_score))

   

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LR: 0.6128604574080212
LDA: 0.7748315103303502
KNN: 0.8920192980517806
CART: 0.8569955437704858
NB: 0.6406290281000258
RF: 0.9194932419990425
XGBoost: 0.8633668471255478


## Result 

Previous result:

LR: 0.6222885132397893
LDA: 0.7628991271682687
KNN: 0.8842116893160976
CART: 0.8704747173424667
NB: 0.5250616874746805
RF: 0.921960740986263
XGBoost: 0.868780613560196

## Random Forest- Model with Evaluation

In [32]:
num_folds = 10
seed = 7
scoring = 'accuracy'

models = []
#models.append(('LR' , LogisticRegression()))
#models.append(('LDA' , LinearDiscriminantAnalysis()))
#models.append(('KNN' , KNeighborsClassifier()))
#models.append(('CART' , DecisionTreeClassifier()))
#models.append(('NB' , GaussianNB()))
#models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
#models.append(('XGBoost', XGBClassifier()))

https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826

In [33]:
results = []
names = []

'''
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) '''

for name, model in models:
    clf = model
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accu_score = accuracy_score(Y_test, Y_pred)
    print(name + ": " + str(accu_score))

    #importing confusion matrix
    from sklearn.metrics import confusion_matrix
    confusion = confusion_matrix(Y_test, Y_pred)
    print('Confusion Matrix\n')
    print(confusion)

    #importing accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(Y_test, Y_pred)))

    print('Micro Precision: {:.2f}'.format(precision_score(Y_test, Y_pred, average='micro')))
    print('Micro Recall: {:.2f}'.format(recall_score(Y_test, Y_pred, average='micro')))
    print('Micro F1-score: {:.2f}\n'.format(f1_score(Y_test, Y_pred, average='micro')))

    print('Macro Precision: {:.2f}'.format(precision_score(Y_test, Y_pred, average='macro')))
    print('Macro Recall: {:.2f}'.format(recall_score(Y_test, Y_pred, average='macro')))
    print('Macro F1-score: {:.2f}\n'.format(f1_score(Y_test, Y_pred, average='macro')))

    print('Weighted Precision: {:.2f}'.format(precision_score(Y_test, Y_pred, average='weighted')))
    print('Weighted Recall: {:.2f}'.format(recall_score(Y_test, Y_pred, average='weighted')))
    print('Weighted F1-score: {:.2f}'.format(f1_score(Y_test, Y_pred, average='weighted')))

    from sklearn.metrics import classification_report
    print('\nClassification Report\n')
    print(classification_report(Y_test, Y_pred, target_names=['class1: minimal', 'class2: moderate', 'class3: substantial', 'class4: widespread']))

RF: 0.9180201082753287
Confusion Matrix

[[16142   411    12     0]
 [  462  3978   392     1]
 [   53   496  4034   102]
 [    4    21   272   773]]

Accuracy: 0.92

Micro Precision: 0.92
Micro Recall: 0.92
Micro F1-score: 0.92

Macro Precision: 0.88
Macro Recall: 0.85
Macro F1-score: 0.86

Weighted Precision: 0.92
Weighted Recall: 0.92
Weighted F1-score: 0.92

Classification Report

                     precision    recall  f1-score   support

    class1: minimal       0.97      0.97      0.97     16565
   class2: moderate       0.81      0.82      0.82      4833
class3: substantial       0.86      0.86      0.86      4685
 class4: widespread       0.88      0.72      0.79      1070

           accuracy                           0.92     27153
          macro avg       0.88      0.85      0.86     27153
       weighted avg       0.92      0.92      0.92     27153

